In [ ]:


from tqdm import tqdm
import numpy as np
from sklearn.ensemble import RandomForestClassifier

import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')

from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
import time
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

plt.style.use(['seaborn-darkgrid'])

In [ ]:
import pandas as pd
train = pd.read_csv('./job_care/train.csv')
test = pd.read_csv('./job_care/test.csv')

In [ ]:
train

In [ ]:
train = train.drop(['id', 'contents_open_dt'], axis=1)
test = test.drop(['id', 'contents_open_dt'], axis=1)

In [ ]:
train

In [ ]:
# 무지성randomforest 하지말고 시각화하고 데이터 살펴보고 뭘 적용할지결정

train = train.astype(np.int)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
x = train.iloc[:,:-1]
y = train['target']

In [ ]:
x = x.astype(np.int)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,y , test_size=0.3 , shuffle=True, stratify=y, random_state=34)

In [ ]:
x_train

In [ ]:
x_train = x_train.astype(np.int)

In [ ]:
# randomforest
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train.values,y_train.values)
y_pred = rf.predict(x_valid)
rf.score(x_train,y_train)
round(rf.score(x_train.values,y_train.values)*100,2)

In [ ]:

plt.plot(y_valid.values[:100], label="answer")
plt.plot(y_pred[:100], label="predict")
plt.legend()

In [ ]:
#GridSearch

rf_param_grid = {
    'n_estimators' :  [100,200,300],
    'max_depth' : [6,8,10,12,20,30,50,70,],
    'min_samples_leaf' : [3,5,7,10],
    'min_samples_split' : [2,3,5,10],
    'n_jobs': [-1]
}
x_train_encoding = x_train.iloc[:,:-2].astype(np.int)

In [ ]:
rf_grid = GridSearchCV(rf, param_grid = rf_param_grid, scoring='accuracy', n_jobs=-1 , verbose =1)
rf_grid.fit(x_train_encoding.values , y_train.values)

rf_grid.best_params_

In [ ]:
# valid set으로 예측을 하고 score 확인 ?

In [ ]:
# LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train.values,y_train.values)
y_pred = logreg.predict(x_valid)


acc_log = round(logreg.score(x_train.values,y_train.values)* 100,2)

In [ ]:

# SelectFromModel

In [ ]:
print("Number of features before selection : {}".format(x_train.shape[1]))

sfm = SelectFromModel(rf , threshold='median', prefit=True)
n_features = sfm.transform(x_train).shape[1]

print('Number of features after seletion : {}'.format(n_features))

selected_vars  = list(x_train.columns[sfm.get_support()])

preds = rf.predict(test.values)
test = test[selected_vars]
preds = rf.predict(test[selected_vars].values)

In [ ]:
train = train[selected_vars + ['target']]

# 버릴 feature 는 버린다 많을 수록 좋은 피쳐가 아니기 때문에
# 제출 submission

In [ ]:
submission = pd.read_csv('./job_care/sample_submission.csv')

submission['target'] = preds
test.columns
submission.to_csv('./baseline.csv', index=False)
baseline = pd.read_csv('./baseline.csv')
baseline['target'].value_counts()

In [ ]:
# f1 score로 train에서 feature 를 가지고 학습한 모델을 test 모델에 적용해서
# 유의미하면 ?


In [ ]:
#PermutationImportance
perm = PermutationImportance(rf, scoring='accuracy', random_state=22).fit(x_valid,y_valid)

In [ ]:
start_time = time.time()
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances : {elapsed_time:.3f} seconds")

In [ ]:
#feature importance
eli5.show_weights(perm , top =20  ,feature_names = x_valid.columns.tolist())
# tree 기반 이므로 얼마나 트리 분할과 밀접한 관련이 있는 지를 본다


# feature 하나하나 마다 shuffle하여 성능 변화 지켜보기 중요한 역할의 feature 라면 모델 서능 떨어질 것


# weight가 양수인갑들은 중요한 값 모델에 큰 영향을 끼친다
# contents attribute d가 중요한 featrue
n_features = len(x_train.columns)

In [ ]:
n_features

In [ ]:
x_train.columns

In [ ]:
model.feature_importances_

In [ ]:
import matplotlib.pyplot as plt

plt.barh(np.arange(n_features), sorted(model.feature_importances_), align='center')
plt.yticks(np.arange(n_features) , x_train.columns)
plt.xlabel('random forest feature importance')
plt.ylabel("Feature")
plt.tight_layout()

In [ ]:
import numpy as np


# 사용자 번호와 컨텐츠 번호는 관련이 없을 듯 한데 제거

# 신경망에 리스트를 주입할 수 없으니 텐서로 변환


In [ ]:
# feature importance
feature_names= test.columns
forest_importances = pd.Series(importances, index=feature_names )

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title('Feature importances using MDI')
ax.set_ylabel("Mena decrease in impurity")
fig.tight_layout()

In [ ]:
# feature importance 가 높은 값에 가중치를 줘서 더 높은 정확도 필요 어떤 콘텐츠 열람하고 시청을 했느냐가 타겟
train

In [ ]:
#회원 선호속성과 컨텐츠 속성과의 연관관계

In [ ]:
# 같은 사용자
# 데이터 시각화 부터 하자
train = train.astype(np.int)

In [ ]:
test = test.astype(np.int)

In [ ]:
# 각각 feature 간의 상관관계
# 대분류중분류소분류가 그래도 타겟과의 상관관계가 그나마 높은 것이 보인다
import seaborn as sns
train

In [ ]:
sns.heatmap(train.corr())

In [ ]:
# 변수 나누기

#
train['d_l_match_yn']

In [ ]:
def reduce_mem_usage(df,verbose=True):
    numerics = ['int16','int32','int64' ,'float16','float32','float64']

    start_mem = df.memory_usage().sum() / 1024**2



    for col in df.columns:
        col_type = df[col].dtype
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                if c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                if c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int8)
                if c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int8)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)

                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        end_mem = df.memory_usage().sum() / 1024 **2
        if verbose: print("Mem , usage decreased to {:5.2f} Mb ({:.1f}% reduction)".format(end_mem, 100 * (start_mem - end_mem) /start_mem))
        return df


In [ ]:
train = reduce_mem_usage(train)

In [ ]:
meta_data = []

In [ ]:
for col_name in train.columns:


    if 'yn' in col_name:
        level = 'binary'
    elif 'attribute' in col_name:
        level = 'nominal'
    elif 'prefer' in col_name:
        level = 'nominal'

    f_dict = {
        'feature_name' : col_name,
        'level' : level
    }

    meta_data.append(f_dict)

In [ ]:
meta_df = pd.DataFrame(meta_data, columns=['feature_name', 'level'])

In [ ]:
meta_df

In [ ]:
# 둘다 오버샘플링 할 필요는 없다 적절한 분포
train['target'].value_counts()

In [ ]:
vars_with_missing =  []

for f in train.columns:
    missings = train[train[f] ==-1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]

        print('Variable {} has {} records : ({:.2f}) with missing values'.format(f , missings , missings_perc))
print('In total , there are {} variables with missing values'.format(len(vars_with_missing)))

In [ ]:
train

In [ ]:
# check cardinality

In [ ]:
v = meta_df[(meta_df.level == 'nominal')].index

In [ ]:
for col in train.columns:
    dist_values = train[col].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(col, dist_values))

In [ ]:
# 변수 시각화

train.dtypes

In [ ]:
# target 이 1인 categorical value 에 대한 percentage

for col in train.columns:
    plt.figure()
    fig , ax = plt.subplots(figsize=(20,10))
    # Calculate the percnetage of target=1 per category value
    cat_perc = train[[col,'target']].groupby([col], as_index=False).mean()
    cat_perc.sort_values(by='target', ascending=False, inplace=True)
    # Bar plot
    # Order the bars decending on target mean
    sns.barplot(ax=ax, x= col, y='target', data=cat_perc, order=cat_perc[col])
    plt.ylabel("target percentage", fontsize=18)
    plt.xlabel(col,fontsize=18)
    plt.tick_params(axis='both', which='major' , labelsize=10)
    plt.show()

In [ ]:
col = meta_df[meta_df.level == 'nominal']['feature_name'].values.tolist()

In [ ]:
col

In [ ]:
col

In [ ]:
train = train.astype(np.int)

In [ ]:
# dummification 하면 값의 수 만큼 컬럼이 늘어난다



pd.get_dummies(train , columns = ['person_attribute_a_1'], drop_first=True)

In [ ]:
# PolynomialFeature
# 각 특서으이 제곱 혹은 그 이상을 추가

In [ ]:
print('Before dummification we have {} variables in train'.format(train.shape[1]))
train = pd.get_dummies(train, columns=col, drop_first=True)
print('After dummification we have {} variables in train'.format(train.shape[1]))

In [ ]:
# 분산이 너무 낮으면 제거한다
from sklearn.feature_selection import VarianceThreshold


selector = VarianceThreshold(threshold=.01)

In [ ]:
train

In [ ]:
# 이진 변수 이고 nomial변수인데 분산이 필요한가?

In [ ]:
selector.fit(train.drop(['target','person_rn','contents_rn']))

In [ ]:
train_select = train.select_dtypes(include=['int'])

In [ ]:
from sklearn.feature_selection import mutual_info_classif

In [ ]:
mutual_info_classif(train_select.values , train.target.values, n_neighbors=3, random_state=17)

In [ ]:
train_select.columns

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(n_estimators=150 , max_depth=8 , min_samples_split=4, max_features=0.2, n_jobs=-1 ,random_state=0)
rf.fit(train,train.target)

In [ ]:
features

In [ ]:
features = train.columns[:-1].values

In [ ]:
import plotly.graph_objs as go

In [ ]:
import plotly.offline as py

In [ ]:
trace = go.Scatter(
    y=rf.feature_importances_,
    x= features,
    mode='markers',
    marker=dict(
        sizemode='diameter',
        sizeref =1 ,
        size= 13,
        color= rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text= features
)
data = [trace]

layout = go.Layout(
    autosize=True,
    title='Random Forest Feature Importance',
    hovermode='closest',
    xaxis = dict(
        ticklen=5,
        showgrid=False ,
        zeroline=False,
        showline=False
    ),
    yaxis = dict(
        title = 'Feature Importance',
        showgrid =False,
        zeroline=False,
        ticklen=5,
        gridwidth=2
    ),
    showlegend=False
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig,filename='scatter2010')

In [ ]:
train

In [ ]:
x, y =list((x) for x in zip(*sorted(zip(rf.feature_importances_,features), reverse=False)))

In [ ]:
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x ,
        colorscale = 'Viridis',
        reversescale=True
    ),
    name='Random Forest Feature Importance',
    orientation='h',
)
layout= dict(
    title='Barplot of Feature importances',
    width= 900, height = 2000,
    yaxis =dict(
        showgrid=False ,
        showline=False,
        showticklabels=True,
    )
)

In [ ]:
# barplot 이 훨씬 시각적으로 와닿는다

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

In [ ]:
from sklearn import tree
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image , ImageDraw , ImageFont
import re

In [ ]:
train.iloc[:,:-1].astype(np.int).drop(['person_rn','contents_rn'],axis=1)

In [ ]:
target = train['target']

In [ ]:
x = train.iloc[:,:-1].astype(np.int)

In [ ]:
target.values

In [ ]:
x

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x,y , test_size=0.3 , shuffle=True, stratify=y, random_state=34)

In [ ]:
x_train

In [ ]:
y_train

In [ ]:
#desiciontreeregressor  를 하면 안되지 .. logisticregresoor도 아니고

In [ ]:
decision_tree = tree.DecisionTreeClassifier(max_depth=3)
decision_tree.fit(x_train, y_train.values)

In [ ]:
y_pred = decision_tree.predict(x_valid)

In [ ]:
import sklearn.metrics as mt

In [ ]:
# accuracy = TP + TN / TP + TN + FP + FN (전체)
accuracy = mt.accuracy_score(y_valid,y_pred)

In [ ]:
# 어느 피쳐가 들어갈 때 성능이 많이 떨어질까?
# 모든 피쳐 조합 ?

# 여부 feature 로만 모델 테스트 해보기


In [ ]:
cols = train.columns.tolist()

In [ ]:
yn_col = []
for col in cols:
    if 'match_yn' in col:
        yn_col.append(col)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
#split마다 고려되는 features의 수 float이면 int(max_features * n_features)값이다

In [ ]:
yn_train = train.loc[:,yn_col].astype(np.int)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(yn_train,target,stratify=target)

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=0)
gb.fit(x_train,y_train)

In [ ]:
(y_valid == gb.predict(x_valid)).mean()

In [ ]:
gb_param_grid = {
    'n_estimators' :  [100,200],
    'max_depth' : [6,8],
    'min_samples_leaf' : [3,5],
    'min_samples_split' : [2,3]
}

In [ ]:
rf_grid = GridSearchCV(gb, param_grid = gb_param_grid, scoring='accuracy', n_jobs=-1 , verbose =1)
rf_grid.fit(x_train,y_train)

rf_grid.best_params_

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=2, max_features=0.2, random_state=0)
gb.fit(x_train,y_train)

In [ ]:
(y_valid == gb.predict(x_valid)).mean()

In [ ]:
attribute_col = []
for col in cols:
    if 'attribute' in col:
        attribute_col.append(col)

In [ ]:
#
attribute_train = train.loc[:,attribute_col].astype(np.int)

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(attribute_train,target,stratify=target)

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=0)
gb.fit(x_train,y_train)

In [ ]:
(y_valid == gb.predict(x_valid)).mean()

In [ ]:
prefer_col = []
for col in cols:
    if 'prefer' in col:
        prefer_col.append(col)

In [ ]:
prefer_train = train.loc[:, prefer_col].astype(np.int)

In [ ]:
# 여부, 선호 , 속성 각각을 트레이닝 해보고 조합도 트레이닝 해본다 ? 별로 좋은 것 같진 않지만

In [ ]:
# 선호는 확실히 모델 성능이 떨어지기는 한다

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(prefer_train,target,stratify=target)

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_leaf=4, max_features=0.2, random_state=0)
gb.fit(x_train,y_train)

In [ ]:
(y_valid == gb.predict(x_valid)).mean()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
target = train['target']

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(yn_train,target,stratify=target)

In [ ]:
x_train = x_train.astype(np.int)

In [ ]:
import lightgbm as lgb

In [ ]:
gb.fit(x_train,y_train)

In [ ]:
# 0.569면 비슷하다 그냥 이것도
(y_valid == gb.predict(x_valid)).mean()

In [ ]:
model = lgb.LGBMClassifier(n_estimators=100, objective='binary', class_weight='balanced'
                           ,learning_rate= 0.05, reg_alpha=0.1, reg_lambda=0.1,
                           subsample=0.8 , n_jobs=-1 , random_state=50)

In [ ]:
model.fit(x_train, y_train , eval_metric='auc' , eval_set=[(x_valid,y_valid) , (x_train,y_train)],
         eval_names=['valid','train'],early_stopping_rounds=100, verbose=200
          )

In [ ]:
x_test = test.loc[: ,yn_col].astype(np.int)

In [ ]:
(gb.predict(x_test) == model.predict(x_test)).mean()

In [ ]:
gb_param_grid = {
    'n_estimators' :  [100,200],
    'max_depth' : [6,8],
    'min_samples_leaf' : [3,5],
    'min_samples_split' : [2,3],
    'n_jobs': [-1]
}
x_train_encoding = x_train.iloc[:,:-2].astype(np.int)

In [ ]:
rf_grid = GridSearchCV(rf, param_grid = rf_param_grid, scoring='accuracy', n_jobs=-1 , verbose =1)
rf_grid.fit(x_train_encoding.values , y_train.values)

rf_grid.best_params_

In [ ]:
GridSearchCV(gb)

In [ ]:
train

In [ ]:
train.columns.value

In [ ]:
train.columns

In [ ]:
train.columns.values.tolist()

In [ ]:

train

In [ ]:
# 덮어쓰기 안되므로 트레이닝 후 수동으로 삭제 후 다시 저자
# Export our trained model as a .dot file
with open("tree1.dot", 'w') as f:
    f = tree.export_graphviz(decision_tree,
                            out_file=f,
                            max_depth=4,
                            impurity= False,
                            feature_names = train.iloc[:,:-1].columns.values.tolist(),
                            class_names =['No' , 'Yes'],
                            rounded =True,
                            filled=True)

In [ ]:
from subprocess import check_call

In [ ]:
# 정확도가 많이 낮네
(y_valid == decision_tree.predict(x_valid)).mean()

In [ ]:
# Convert .dot to .png to allow display in web notebook
check_call(['dot', '-Tpng', 'tree1.dot' , '-o','tree1.png'])

In [ ]:
# Annotating chart with PIL
img = Image.open("tree1.png")
draw = ImageDraw.Draw(img)
img.save("sample-out.png")
PImage("sample-out.png")

In [ ]:
# Annotating chart with PIL
img = Image.open("tree1.png")
draw = ImageDraw.Draw(img)
img.save('sample-out.png')
PImage("sample-out.png")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split()

In [ ]:
import os
os.getcwd()

In [ ]:
print(mf)

In [ ]:
v = train

In [ ]:
# feature engineering
print('Before dummification we have {} variables in train')

In [ ]:
# category 2 is not twice the value of category 1

In [ ]:
train['person_prefer_d_1']

In [ ]:
import seaborn as sns

In [ ]:
# 같은 컨텐츠
train['contents_rn'].value_counts()

In [ ]:

import numpy as np
import pandas as pd
import torch

In [397]:
from torch import nn
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

In [398]:
train_df = pd.read_csv('./job_care/train.csv')
test_df = pd.read_csv('./job_care/test.csv')

In [399]:
train = train_df[train_df['contents_open_dt'].apply(lambda x: pd.Timestamp(x).month)<11].copy()

In [400]:
val = train_df[train_df['contents_open_dt'].apply(lambda x: pd.Timestamp(x).month)==11].copy()

In [401]:
#preprocessing
test= test_df.copy()

In [402]:
for df in [train,val,test]:
    df.drop(['contents_open_dt','contents_rn','id','person_rn','person_prefer_f','person_prefer_g'],axis=1,inplace=True)

In [403]:
columns = sorted(test.columns)
train = train[columns+ ['target']] *1
val = val[columns+['target']]*1
test = test[columns] * 1

In [404]:
cat_idxs = []
cat_dims = []

In [405]:
for idx, col in enumerate(train.columns):
    if 'match' not in col and col != 'target':
        le = LabelEncoder()
        le.fit(train_df[col].values)
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        train[col] = train[col].apply(lambda x: le_dict.get(x,len(le_dict)))
        val[col] = val[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        test[col] = test[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        cat_idxs.append(idx)
        cat_dims.append(len(le_dict)+1)

In [406]:
X_train = train.drop('target',axis=1).values
y_train = train['target'].values
X_val = val.drop('target', axis=1).values
y_val = val['target'].values
X_test = test.values
eval_set = (X_val, y_val)

In [386]:
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=5,
                       optimizer_fn=torch.optim.AdamW,# Any optimizer work here
                       mask_type='entmax',#"spaesemax")
                       )

Device used : cpu


In [387]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True
    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:,1]>0.5)* 1)
        return score

In [388]:
y_valid.values

array([1, 0, 1, ..., 1, 1, 1])

In [389]:
#이건 어떻게 하길래 더 높은 정확도가 나올까
clf.fit(
    X_train= x_train.values, y_train=y_train.values,
    eval_set=[(x_train.values, y_train.values), (x_valid.values, y_valid.values)],
    eval_name=['train', 'val'],
    eval_metric=['logloss','f1'],
    max_epochs=100, patience=2,
    batch_size=1024,
    virtual_batch_size=256,
    num_workers=1,
    drop_last=False,
)

IndexError: index 6 is out of bounds for dimension 0 with size 6

In [363]:
preds = clf.predict_proba(x_test)
preds = (preds[:,1]>0.5)*1

AttributeError: 'TabNetClassifier' object has no attribute 'network'

In [362]:
submission = pd.read_csv('./job_care/sample_submission.csv')
submission['target'] = preds

TypeError: read_csv() missing 1 required positional argument: 'filepath_or_buffer'

In [390]:

import numpy as np
import pandas as pd
import torch

from torch import nn
from pytorch_tabnet.tab_model  import TabNetClassifier
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

train_df = pd.read_csv('./job_care/train.csv')
test_df = pd.read_csv('./job_care/test.csv')

train = train_df[train_df['contents_open_dt'].apply(lambda x: pd.Timestamp(x).month)<11].copy()
val = train_df[train_df['contents_open_dt'].apply(lambda x: pd.Timestamp(x).month)==11].copy()
test = test_df.copy()

In [391]:
for df in [train,val,test]:
    df.drop(['contents_open_dt','contents_rn','id','person_rn','contents_open_dt','person_prefer_f','person_prefer_g'],axis=1,inplace=True)

In [392]:
columns = sorted(test.columns)
train = train[columns+['target']]*1
val = val[columns+['target']]*1
test = test[columns]*1

cat_idxs = []
cat_dims = []
for idx, col in enumerate(train.columns):
    if 'match' not in col and col!='target':
        le = LabelEncoder()
        le.fit(train_df[col].values)
        le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
        train[col] = train[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        val[col] = val[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        test[col] = test[col].apply(lambda x: le_dict.get(x, len(le_dict)))
        cat_idxs.append(idx)
        cat_dims.append(len(le_dict)+1)

In [393]:
X_train = train.drop('target',axis=1).values
y_train = train['target'].values
X_val = val.drop('target',axis=1).values
y_val = val['target'].values
X_test = test.values
eval_set = (X_val,y_val)

In [394]:
clf = TabNetClassifier(cat_idxs=cat_idxs,
                       cat_dims=cat_dims,
                       cat_emb_dim=3,
                       optimizer_fn=torch.optim.AdamW, # Any optimizer works here
                       mask_type='entmax', # "sparsemax",
                      )

Device used : cpu


In [395]:
class F1_Score(Metric):
    def __init__(self):
        self._name = "f1"
        self._maximize = True

    def __call__(self, y_true, y_score):
        score = f1_score(y_true, (y_score[:, 1]>0.5)*1)
        return score

In [396]:
X_train
y_train

epoch 0  | loss: 0.68455 | train_logloss: 0.67565 | train_f1: 0.6035  | val_logloss: 0.67816 | val_f1: 0.59826 |  0:01:19s


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x1425d1b80>
Traceback (most recent call last):
  File "/Users/seongminpark/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/Users/seongminpark/PycharmProjects/pythonProject/venv/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1301, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/multiprocessing/popen_fork.py", line 44, in wait
    if not wait([self.sentinel], timeout):
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python

KeyboardInterrupt: 

In [ ]:
x_valid
y_valid

In [ ]:
clf.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'val'],
    eval_metric=['logloss','f1'],
    max_epochs=100 , patience=2,
    batch_size=1024,
    virtual_batch_size=256,
    num_workers=1,
    drop_last=False,
)

preds = clf.predict_proba(X_test)
preds = (preds[:,1]>0.5)*1

In [408]:

train = pd.read_csv('./JobCare_data/train.csv')

In [412]:
train[['person_attribute_a_1','person_attribute_b','person_attribute_b','contents_attribute_e']]

,person_attribute_a_1,person_attribute_b,person_attribute_b,contents_attribute_e
0,4,3,3,4
1,3,4,4,4
2,0,3,3,4
3,0,2,2,3
4,3,4,4,4
...,...,...,...,...
501946,1,2,2,5
501947,6,2,2,4
501948,7,4,4,7
501949,1,2,2,4


In [420]:
attr_a_1_mean_encode = train.groupby('person_attribute_a_1')["target"].mean()

In [434]:

if 'attribute' in col:
    name = 'person_attribute_a_1'['person_attribute_a_1'.index('attribute') + len('attribute')+1:]
elif 'prefer' in col:
    name =

'a_1'

In [ ]:
nominal_cols[0]

In [437]:
nominal_cols = ['person_attribute_a_1','person_attribute_b','person_prefer_e','contents_attribute_e']

In [446]:
# 오버피팅이 자주 발생하는 mean encoding 이므로 cross validation 과 정규화 같이 사용한다=
# 변환 하고자 하는 범주형 변수 선택
# 범주형 변수 그룹화 -> 타깃 변수 총합 합계
# 범주형 변수 그룹화 타깃 빈도수 합계
# 총합을 카운트로 나누고 본래 범주 값에 업데이트
# 여러가지 방법으로 적용 가능하다
# 비슷한 범주 사이에 있는 관계 표현 특징, 범주와 타깃사이에만 국한된다
# 범주가 많은 경우 이 방법은 데이터를 훨씬 더 단순화 한다

for col in nominal_cols:
    if 'attribute' in col:
        name = col[col.index('attribute') + len('attribute')+1:]
        var_name = 'attr_{}_mean_encode'.format(name)
        locals()[var_name] = train.groupby(col)["target"].mean()
    elif 'prefer' in col:
        name = col[col.index('prefer') + len('prefer')+1:]
        var_name = 'prefer_{}_mean_encode'.format(name)
        locals()[var_name] = train.groupby(col)["target"].mean()

    train.loc[:,var_name] = train[col].map(locals()[var_name])

In [ ]:
target = train.target
train = train.drop('target',axis=1)
x_train, x_valid, y_train, y_valid = train_test_split(train,target , test_size=0.3 , shuffle=True, stratify=y, random_state=34)

rf= RandomForestClassifier(n_estimators=150 , max_depth=8 , min_samples_split=4, max_features=0.2, n_jobs=-1 ,random_state=0)

rf.fit(x_train,y_train)
# 과적합일진 모르지만 그래도많이올랐따 만족할 수 없음
(y_valid == rf.predict(x_valid)).mean()

In [ ]:
# 위와 다르게 smooth한 평균을 계산하고 적용하는 방법
# 1. 평균을 계산
Mean = train['target'].mean()
weight = 100
for col in nominal_cols:

    if 'attribute' in col:
        name = col[col.index('attribute') + len('attribute')+1:]
        var_name = 'attr_{}_mean_encode'.format(name)

    elif 'prefer' in col:
        name = col[col.index('prefer') + len('prefer')+1:]
        var_name = 'prefer_{}_mean_encode'.format(name)

    # 2. 각 그룹에 대한 값들의 빈도와 평균을 계산
    Agg = train.groupby(col)['target'].agg(['count','mean'])
    counts = Agg['count']
    #%%
    means = Agg['mean']

    # 3. 'smooth'한 평균을 계산
    smooth = (counts * means + weight * means) / (counts+weight)
    print(smooth)

    # smooth한 평균에 따라 각 값을 대체하는 것
    train.loc[:,'smooth_'+var_name] = train[col].map(smooth)

In [479]:
train=train.drop(['smoothattr_a_1_mean_encode',	'smoothattr_b_mean_encode'	,'smoothprefer_e_mean_encode'	,'smoothattr_e_mean_encode'],axis=1)

In [ ]:
train  =train.drop(['attr_a_1_mean_encode','attr_b_mean_encode','prefer_e_mean_encode','attr_e_mean_encode','target'],axis=1)

In [474]:
x_train, x_valid, y_train, y_valid = train_test_split(train,target , test_size=0.3 , shuffle=True, stratify=target, random_state=34)

In [475]:
rf= RandomForestClassifier(n_estimators=150 , max_depth=8 , min_samples_split=4, max_features=0.2, n_jobs=-1 ,random_state=0)

rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=8, max_features=0.2, min_samples_split=4,
                       n_estimators=150, n_jobs=-1, random_state=0)

In [476]:
# 과적합일진 모르지만 그래도많이올랐따 만족할 수 없음
(y_valid == rf.predict(x_valid)).mean()

0.5971670673236555

In [ ]:
train = train.drop(['id','contents_open_dt','person_rn','contents_rn'],axis=1)

In [453]:
target = train.target

In [454]:
train = train.drop('target',axis=1)

In [455]:
x_train, x_valid, y_train, y_valid = train_test_split(train,target , test_size=0.3 , shuffle=True, stratify=y, random_state=34)

In [ ]:
rf= RandomForestClassifier(n_estimators=150 , max_depth=8 , min_samples_split=4, max_features=0.2, n_jobs=-1 ,random_state=0)

In [456]:
rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=8, max_features=0.2, min_samples_split=4,
                       n_estimators=150, n_jobs=-1, random_state=0)

In [458]:
# 과적합일진 모르지만 그래도많이올랐따 만족할 수 없음
(y_valid == rf.predict(x_valid)).mean()

0.5971670673236555

In [481]:
####### Weight of Evidence Encoding (WoE)

# 각 범주가 target = 1일 확률 (좋은(good) = 1 일 확률) 을 계산한다.

In [482]:
train['target']= target
woe_df = train.groupby('person_attribute_a_1')['target'].mean()

In [484]:
woe_df = pd.DataFrame(woe_df)

In [485]:
# 칼럼의 이름을 "good" 으로 바꾸어 좀 더 이해하기 쉽게 한다
woe_df = woe_df.rename(columns = {'target':'good'})

In [486]:
woe_df['bad'] = 1 -woe_df.good
#분모에 최소한의 값을 더하여 0으로 나뉘는 일을 막는다
woe_df['bad'] = np.where(woe_df['bad']  == 0, 1e-6,woe_df['bad'])
#WoE를 계산한다
woe_df['WoE'] = np.log(woe_df.good/woe_df.bad)

In [487]:
# 가중치 인코딩이니 좀 더 내가 원했듯이 별 차이 없는 (0이어도 타겟값이 1:1비율로 퍼져있는)
# 그런 변수는 woe값이 아주 낮게 나오는게 아주맘에든다 바로이거다 전체구조를 파악하고 기억할 생각을 안해서 그렇지
# 하나씩 기억을 하고 이루다 개발하면서 로그시스템도 척척 restfulapi 스프링 서비스 구조를 알다보니
# 분명히 이건 이걸껀데 하면서 하나의 웹사이트에서 본 지식이 이해가 안가면 다른 웹사이트에서 본 지식과 내가 알고 있는것을 합쳐서
# 보니 이해할 수 있었다는 것을 얻었다는게 아주중요하다 오늘 WoE 값이 내가 원하던 값
woe_df

,good,bad,WoE
person_attribute_a_1,,,
0,0.493002,0.506998,-0.027992
1,0.497688,0.502312,-0.009249
2,0.496387,0.503613,-0.014453
3,0.486858,0.513142,-0.052580
4,0.483610,0.516390,-0.065585
5,0.496374,0.503626,-0.014503
6,0.515074,0.484926,0.060315
7,0.565771,0.434229,0.264617


In [ ]:
locals()['attr_a_1_mean_encode']

In [440]:
locals()['attr_b_mean_encode']

person_attribute_b
0    0.466667
1    0.544433
2    0.503328
3    0.487071
4    0.500850
5    0.552997
Name: target, dtype: float64

In [441]:
locals()['prefer_e_mean_encode']

person_prefer_e
0     0.488677
1     0.516393
2     0.502798
3     0.479206
4     0.500543
5     0.512256
6     0.531888
7     0.543571
8     0.580604
9     0.591989
10    0.652376
11    0.579890
Name: target, dtype: float64

In [443]:
locals()['attr_e_mean_encode']

contents_attribute_e
0     0.238049
1     0.353736
2     0.426174
3     0.498611
4     0.504105
5     0.506072
6     0.510225
7     0.528446
8     0.522676
9     0.539648
10    0.555721
11    0.401734
Name: target, dtype: float64

In [445]:
train

,id,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,...,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,person_rn,contents_rn,contents_open_dt,target,attr_a_1_mean_encode
0,0,True,True,True,False,False,False,1,4,3,...,1608,275,1,4,139,618822,354805,2020-01-17 12:09:36,1,0.483610
1,1,False,False,False,True,True,False,1,3,4,...,1608,275,1,4,133,571659,346213,2020-06-18 17:48:52,0,0.486858
2,2,False,False,False,True,False,False,2,0,3,...,1600,94,1,4,53,399816,206408,2020-07-08 20:00:10,0,0.493002
3,3,False,False,False,True,False,False,2,0,2,...,1608,275,5,3,74,827967,572323,2020-01-13 18:09:34,0,0.493002
4,4,True,True,True,False,False,False,1,3,4,...,1608,275,1,4,74,831614,573899,2020-03-09 20:39:22,0,0.486858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,501946,False,False,False,True,False,False,1,1,2,...,354,147,1,5,65,503156,285850,2020-03-13 12:55:52,1,0.497688
501947,501947,True,True,False,True,False,False,1,6,2,...,163,120,1,4,142,676255,456996,2020-01-20 11:51:51,1,0.515074
501948,501948,True,True,True,True,False,False,1,7,4,...,438,147,2,7,65,484528,293258,2020-08-05 17:27:24,1,0.565771
501949,501949,True,False,False,True,False,False,1,1,2,...,660,147,3,4,259,456330,273797,2020-06-15 09:23:21,1,0.497688


In [ ]:
attr_a_1_mean_encode

In [ ]:
train.loc[:,''] =



In [ ]:
# 각 범주에 대해 , 타깃 = 1 인 확률 (좋은(Good) = 1 일 확률을 찾는다)을 찾는다



In [488]:
pr_df = train.groupby("person_attribute_a_1")['target'].mean()

In [490]:
pr_df

person_attribute_a_1
0    0.493002
1    0.497688
2    0.496387
3    0.486858
4    0.483610
5    0.496374
6    0.515074
7    0.565771
Name: target, dtype: float64

In [491]:
pr_df = pd.DataFrame(pr_df)

In [492]:
pr_df = pr_df.rename(columns = {'target' : 'good'})

In [493]:
pr_df['bad'] = 1 - pr_df.good


In [494]:
pr_df

,good,bad
person_attribute_a_1,,
0,0.493002,0.506998
1,0.497688,0.502312
2,0.496387,0.503613
3,0.486858,0.513142
4,0.483610,0.516390
5,0.496374,0.503626
6,0.515074,0.484926
7,0.565771,0.434229


In [496]:
# 분모에 최소한의 값을 더하여 0으로 나뉘는 일을 막는다

pr_df['bad'] = np.where(pr_df['bad'] == 0 , 1e-6 ,  pr_df['bad'])

In [497]:
#확률 비율을 계산한다

pr_df['PR'] = pr_df.good/pr_df.bad

In [498]:
pr_df

,good,bad,PR
person_attribute_a_1,,,
0,0.493002,0.506998,0.972396
1,0.497688,0.502312,0.990793
2,0.496387,0.503613,0.985651
3,0.486858,0.513142,0.948778
4,0.483610,0.516390,0.936520
5,0.496374,0.503626,0.985602
6,0.515074,0.484926,1.062171
7,0.565771,0.434229,1.302932


In [499]:
train.loc[:,'PR_Encode'] = train['person_attribute_a_1'].map(pr_df['PR'])

In [500]:
train

,d_l_match_yn,d_m_match_yn,d_s_match_yn,h_l_match_yn,h_m_match_yn,h_s_match_yn,person_attribute_a,person_attribute_a_1,person_attribute_b,person_prefer_c,...,contents_attribute_j,contents_attribute_c,contents_attribute_k,contents_attribute_l,contents_attribute_d,contents_attribute_m,contents_attribute_e,contents_attribute_h,target,PR_Encode
0,True,True,True,False,False,False,1,4,3,5,...,2,1,2,1608,275,1,4,139,1,0.936520
1,False,False,False,True,True,False,1,3,4,1,...,1,1,2,1608,275,1,4,133,0,0.948778
2,False,False,False,True,False,False,2,0,3,5,...,2,1,1,1600,94,1,4,53,0,0.972396
3,False,False,False,True,False,False,2,0,2,5,...,1,1,2,1608,275,5,3,74,0,0.972396
4,True,True,True,False,False,False,1,3,4,5,...,2,1,2,1608,275,1,4,74,0,0.948778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501946,False,False,False,True,False,False,1,1,2,2,...,1,1,2,354,147,1,5,65,1,0.990793
501947,True,True,False,True,False,False,1,6,2,1,...,2,1,2,163,120,1,4,142,1,1.062171
501948,True,True,True,True,False,False,1,7,4,1,...,1,1,2,438,147,2,7,65,1,1.302932
501949,True,False,False,True,False,False,1,1,2,1,...,1,1,2,660,147,3,4,259,1,0.990793
